<a href="https://colab.research.google.com/github/gokhangunal/AB_Testing_I/blob/main/AB_Testing_I.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import itertools
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
!pip install statsmodels
import statsmodels.stats.api as sms
from scipy.stats import ttest_1samp, shapiro, levene, ttest_ind, mannwhitneyu, \
    pearsonr, spearmanr, kendalltau, f_oneway, kruskal
from statsmodels.stats.proportion import proportions_ztest

In [18]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 10)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [8]:
# Excel dosyanızın yolu
excel_dosyasi = 'ab_testing.xlsx'

# Control group sayfasını oku
control_group = pd.read_excel(excel_dosyasi, sheet_name='Control Group')

# Test group sayfasını oku
test_group = pd.read_excel(excel_dosyasi, sheet_name='Test Group')

# Şimdi `control_group` ve `test_group` DataFrame'leri üzerinde istediğiniz işlemleri yapabilirsiniz.


In [9]:
control_group.head(5)

,Impression,Click,Purchase,Earning
0,82529.45927,6090.07732,665.21125,2311.27714
1,98050.45193,3382.86179,315.08489,1742.80686
2,82696.02355,4167.96575,458.08374,1797.82745
3,109914.40040,4910.88224,487.09077,1696.22918
4,108457.76263,5987.65581,441.03405,1543.72018


In [10]:
test_group.head(5)

,Impression,Click,Purchase,Earning
0,120103.50380,3216.54796,702.16035,1939.61124
1,134775.94336,3635.08242,834.05429,2929.40582
2,107806.62079,3057.14356,422.93426,2526.24488
3,116445.27553,4650.47391,429.03353,2281.42857
4,145082.51684,5201.38772,749.86044,2781.69752


In [24]:
control_group.describe().T

,count,mean,std,min,25%,50%,75%,max
Impression,40.00,101711.45,20302.16,45475.94,85726.69,99790.70,115212.82,147539.34
Click,40.00,5100.66,1329.99,2189.75,4124.30,5001.22,5923.80,7959.13
Purchase,40.00,550.89,134.11,267.03,470.10,531.21,637.96,801.80
Earning,40.00,1908.57,302.92,1253.99,1685.85,1975.16,2119.80,2497.30


In [23]:
test_group.describe().T

# Çıkarım:
# Reklam Görüntüleme ortalaması test grubunda daha yüksek.
# Görüntülenen reklam tıklama ortalaması kontrol grubunda daha yüksek.
# Tıklanan reklamlar sonrası satın alınan ürün sayısı test grubunda daha yüksek.
# Satın alınan ürünler sonrası elde edilen kazanç miktari test grubunda daha yüksek.
# Kontrol grubu maximum bidding yani eski yöntemi temsil etmekte
# Test grubu average bidding yani yeni yöntemi temsil etmektedir.

,count,mean,std,min,25%,50%,75%,max
Impression,40.00,120512.41,18807.45,79033.83,112691.97,119291.30,132050.58,158605.92
Click,40.00,3967.55,923.10,1836.63,3376.82,3931.36,4660.50,6019.70
Purchase,40.00,582.11,161.15,311.63,444.63,551.36,699.86,889.91
Earning,40.00,2514.89,282.73,1939.61,2280.54,2544.67,2761.55,3171.49


In [27]:
# `control_group` DataFrame'ine 'group' adında yeni bir kolon ekle ve 'control' olarak ayarla
control_group['group'] = 'control'

# `test_group` DataFrame'ine 'group' adında yeni bir kolon ekle ve 'test' olarak ayarla
test_group['group'] = 'test'

# Dikey olarak birleştir (alt alta) ve 'group' kolonunu kullanarak hangi grubun verileri olduğunu belirt
birlesik_df = pd.concat([control_group, test_group], axis=0)

# Birleştirme işleminden sonra indeksleri sıfırla
birlesik_df.reset_index(drop=True, inplace=True)

# Birleştirilmiş DataFrame'in ilk birkaç satırını göster
print(birlesik_df.head())

   Impression   Click  Purchase  Earning    group
0    82529.46 6090.08    665.21  2311.28  control
1    98050.45 3382.86    315.08  1742.81  control
2    82696.02 4167.97    458.08  1797.83  control
3   109914.40 4910.88    487.09  1696.23  control
4   108457.76 5987.66    441.03  1543.72  control


In [ ]:
######################################################
# AB Testing (Bağımsız İki Örneklem T Testi)
######################################################

# 1. Hipotezleri Kur
# 2. Varsayım Kontrolü
#   - 1. Normallik Varsayımı (Değerler normal dağılıyor mu?)
#   - 2. Varyans Homojenliği (Değerlerin varyansı birbiriyle benzeşiyor mu?)
# 3. Hipotezin Uygulanması
#   - 1. Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
#   - 2. Varsayımlar sağlanmıyorsa mannwhitneyu testi (non-parametrik test)
# 4. p-value değerine göre sonuçları yorumla
# Not:
# - Normallik sağlanmıyorsa direkt 2 numara uygulanır. Varyans homojenliği sağlanmıyorsa 1 numaraya arguman girilir (Bak varyanslar homojen değil denir).
# - Normallik incelemesi öncesi aykırı değer incelemesi ve düzeltmesi yapmak faydalı olabilir.

In [ ]:
############################
# Uygulama 1: Uygulamaya konan yeni bidding sistemi ile eski bidding sistemi arasında istatistiksel olarak anlamlı bir fark var mı?
############################

In [ ]:
############################
# 1. Hipotezi Kur
############################

# H0: M1 = M2
# H1: M1 != M2

############################
# 2. Varsayım Kontrolü
############################

# Normallik Varsayımı
# Varyans Homojenliği

############################
# Normallik Varsayımı
############################

# H0: Normal dağılım varsayımı sağlanmaktadır.
# H1:..sağlanmamaktadır.

In [39]:
birlesik_df.groupby("group").agg({"Purchase": "mean"})

,Purchase
group,
control,550.89
test,582.11


In [40]:
#Tekrar bir mini hipotez testine girişiyoruz. H0 normal dağılıyor diyor. H1 ise normal dağılmıyor diyor. Shapiro metodu ile test ediyoruz.
# Bunu önce kontrol grubu için test ediyorum
# p-value < ise 0.05'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.
test_stat, pvalue = shapiro(birlesik_df.loc[birlesik_df["group"] == "control", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891


In [41]:
# HO reddedilemez. Dolayısıyla normal bir dağılım sağlanmıştır.

In [42]:
#Tekrar bir mini hipotez testine girişiyoruz. H0 normal dağılıyor diyor. H1 ise normal dağılmıyor diyor. Shapiro metodu ile test ediyoruz.
# Bunu şimdi test grubu için test ediyorum
# p-value < 0.05 ise HO RED.
# p-value < 0.05 değilse H0 REDDEDILEMEZ.
test_stat, pvalue = shapiro(birlesik_df.loc[birlesik_df["group"] == "test", "Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9589, p-value = 0.1541


In [43]:
# HO reddedilemez. Dolayısıyla normal bir dağılım sağlanmıştır.

In [44]:
# NORMAL DAĞILIM VARSAYIMI SAĞLANMIŞTIR.

# Şayet normal dağılım şartı sağlanmasaydı varyans testini yapmayıp otomatik olarak
# Non-Parametrik bir teste geçecektim.

In [45]:
############################
# Varyans Homojenligi Varsayımı
############################
# Varyans, verilerin aritmetik ortalamadan sapmalarının karelerinin toplamıdır. Yani standart sapmanın karekök alınmamış halidir.
# H0: Varyanslar Homojendir
# H1: Varyanslar Homojen Değildir

In [46]:
test_stat, pvalue = levene(birlesik_df.loc[birlesik_df["group"] == "control", "Purchase"],
                           birlesik_df.loc[birlesik_df["group"] == "test", "Purchase"])

# Varyans homojenliği testini yapmak için levene fonksiyonunu kullanıyoruz.

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


In [47]:
# p-value < ise 0.05 'ten HO RED.
# p-value < değilse 0.05 H0 REDDEDILEMEZ.

# H0 reddedilemez. Dolayısıyla varyanslar homojendir.

In [48]:
############################
# Varsayımlar sağlanıyorsa bağımsız iki örneklem t testi (parametrik test)
############################

In [50]:
test_stat, pvalue = ttest_ind(birlesik_df.loc[birlesik_df["group"] == "control", "Purchase"],
                              birlesik_df.loc[birlesik_df["group"] == "test", "Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))


# ttest_ind fonksiyonu diyor ki:
#   1. İki varsayım da sağlanıyorsa beni kullanabilirsin.
#   2. Yalnızca normal dağılım varsayımı sağlanıyorsa beni kullanabilirsin.
#   3. Varyans homojenliği varsayımı sağlanmıyorsa ancak ve ancak equal_var = False ile beni kullanabilirsin.
#     Böylece arka tarafta Welch Testi yapılacaktır.!!!!!!!!

# p-value < 0.05 ise HO RED.
# p-value < 0.05 değilse H0 REDDEDILEMEZ.

Test Stat = -0.9416, p-value = 0.3493


In [ ]:
# HO reddedilemez. Yani iki grup ortalamaları arasında istatiksel olarak anlamlı bir fark yoktur ifadesi doğrulanmıştır.
# Yani average bidding yönteminin uygulanması ile birlikte istatistiksel olarka anlamlı bir satın alım farkı yaratılamamış ve
# bu yeni yöntem firmamıza daha fazla satın alım getirmiştir diyemeyiz.

In [ ]:
# Test Stat: Bu değer, iki grubun ortalama değerleri arasındaki farkın,
# bu farkın standart hatasına bölünmesiyle hesaplanır. Negatif bir T istatistiği değeri,
# test grubunun ortalama değerinin kontrol grubunun ortalama değerinden daha küçük olduğunu gösterir.
# Bu kadar büyük (mutlak değerde) bir T istatistiği, iki grup arasındaki ortalama farkın oldukça belirgin olduğunu gösterir.

In [ ]:
# Diğer istatistik kolonları da incelenmeli anlamlı bir fark varsa ortaya konmalıdır. Tek bir kolona göre yeni yöntemin verimsiz olduğu
# öne sürülemez.